## Case Study 1

In [3]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import cmath
import math
import time
import pandas
import scipy.integrate as integrate
import cvxpy as cvx



In [4]:
def Characteristic(S,r,q,sigma,t,u):
    phi = np.exp(1j*(np.log(S) + (r-q-0.5*sigma**2)*t)*u-0.5*sigma**2*u**2*t)
    return phi

In [11]:
def Option_Premium_FFT(S,r,q,sigma,T,K,n,alpha,eta):
    N = 2**n
    lbda = 2*math.pi/(N*eta)
    Beta_0 = np.log(K)-0.5*N*lbda 
    x = np.zeros(N)
    nu = np.zeros(N)
    for i in range(N):
        nu[i] = i*eta
    df = np.exp(-r*T)
    

    for i in range(N):
        if i == 0:
            PsiJ = Characteristic(S,r,q,sigma,T,nu[i]-(alpha+1)*1j)
            weightJ = 2*(alpha+1j*nu[i])*(alpha+1j*nu[i]+1)
            x[i] = eta*df*np.exp(-1j*Beta_0*nu[i])*PsiJ/weightJ
        elif i != 0:
            PsiJ = Characteristic(S,r,q,sigma,T,nu[i]-(alpha+1)*1j)
            weightJ = (alpha+1j*nu[i])*(alpha+1j*nu[i]+1)
            x[i] = eta*df*np.exp(-1j*Beta_0*nu[i])*PsiJ/weightJ
        #print(Characteristic(S,r,q,sigma,T,nu[i]-(alpha+1)*1j))
    #print('x', x)
    y = np.fft.fft(x)
    #print('y', y)
    k = np.zeros(N)
    C_T = np.zeros(N)
    for i in range(N):
        k[i] = Beta_0 + i*lbda
        mult = np.exp(-alpha*k[i])/math.pi
        C_T[i] = mult*np.real(y[i])
    
    data = pandas.DataFrame({"Strike": np.exp(k),"Option Premium":C_T})
    print("Option via FFT: For Strike Price %s and the given parameters, we get that the Option Premium is: " %K)
    print(str(round(data.iloc[int(0.5*N)]["Option Premium"],4)))
    

In [12]:
#Option_Premium_FFT(S,r,q,sigma,T,K,n,alpha,eta)
#For K = 2000
Option_Premium_FFT(1900,0.02,0.0187,0.36,0.25,2000,9,0.4,0.25)
Option_Premium_FFT(1900,0.02,0.0187,0.36,0.25,2000,11,1,0.25)
Option_Premium_FFT(1900,0.02,0.0187,0.36,0.25,2000,13,1.4,0.25)
Option_Premium_FFT(1900,0.02,0.0187,0.36,0.25,2000,11,3,0.25)

Option via FFT: For Strike Price 2000 and the given parameters, we get that the Option Premium is: 
95.3281
Option via FFT: For Strike Price 2000 and the given parameters, we get that the Option Premium is: 
95.2467
Option via FFT: For Strike Price 2000 and the given parameters, we get that the Option Premium is: 
95.2467
Option via FFT: For Strike Price 2000 and the given parameters, we get that the Option Premium is: 
95.2467


In [5]:
#For K = 2100
Option_Premium_FFT(1900,0.02,0.0187,0.36,0.25,2100,9,0.4,0.25)
Option_Premium_FFT(1900,0.02,0.0187,0.36,0.25,2100,11,1,0.25)
Option_Premium_FFT(1900,0.02,0.0187,0.36,0.25,2100,13,1.4,0.25)
Option_Premium_FFT(1900,0.02,0.0187,0.36,0.25,2100,11,3,0.25)


Option via FFT: For Strike Price 2100 and the given parameters, we get that the Option Premium is: 
64.916
Option via FFT: For Strike Price 2100 and the given parameters, we get that the Option Premium is: 
64.8346
Option via FFT: For Strike Price 2100 and the given parameters, we get that the Option Premium is: 
64.8346
Option via FFT: For Strike Price 2100 and the given parameters, we get that the Option Premium is: 
64.8346


In [6]:
#For K = 2200
Option_Premium_FFT(1900,0.02,0.0187,0.36,0.25,2200,9,0.4,0.25)
Option_Premium_FFT(1900,0.02,0.0187,0.36,0.25,2200,11,1,0.25)
Option_Premium_FFT(1900,0.02,0.0187,0.36,0.25,2200,13,1.4,0.25)
Option_Premium_FFT(1900,0.02,0.0187,0.36,0.25,2200,11,3,0.25)


Option via FFT: For Strike Price 2200 and the given parameters, we get that the Option Premium is: 
43.0286
Option via FFT: For Strike Price 2200 and the given parameters, we get that the Option Premium is: 
42.9472
Option via FFT: For Strike Price 2200 and the given parameters, we get that the Option Premium is: 
42.9472
Option via FFT: For Strike Price 2200 and the given parameters, we get that the Option Premium is: 
42.9472


 Fractional Fast Fourier Transform

In [37]:
def Option_Premium_FrFFT(S,r,q,sigma,T,K,n,alpha,eta,lbda):
    N = 2**n
    gamma = (eta*lbda)/(2*math.pi)
    Beta_0 = np.log(K)-0.5*N*lbda 
    x = np.zeros(N)
    nu = np.zeros(N)
    for i in range(N):
        nu[i] = i*eta
    df = np.exp(-r*T)
    
    
    for i in range(N):
        if i == 0:
            PsiJ = Characteristic(S,r,q,sigma,T,nu[i]-(alpha+1)*1j)
            weightJ = 2*(alpha+1j*nu[i])*(alpha+1j*nu[i]+1)
            x[i] = eta*df*np.exp(-1j*Beta_0*nu[i])*PsiJ/weightJ
        elif i != 0:
            PsiJ = Characteristic(S,r,q,sigma,T,nu[i]-(alpha+1)*1j)
            weightJ = (alpha+1j*nu[i])*(alpha+1j*nu[i]+1)
            x[i] = eta*df*np.exp(-1j*Beta_0*nu[i])*PsiJ/weightJ
    
    y = np.zeros(2*N, dtype = np.complex)
    z = np.zeros(2*N, dtype = np.complex)
    for i in range(2*N):
        if i < N :
            y[i] = np.exp(-1j*math.pi*gamma*(i**2))*x[i]
            z[i] = np.exp(gamma*math.pi*1j*(i**2))
        elif i >= N:
            z[i] = np.exp(gamma*math.pi*1j*((2*N-i-1)**2))
    
    y_hat = np.fft.fft(y)
    #print(y_hat)
    z_hat = np.fft.fft(z)
    #£print(z_hat)
    epsilon_hat = np.fft.ifft(y_hat*z_hat)
    
    
    k = np.zeros(N)
    C_T = np.zeros(N)
    for i in range(N):
        k[i] = Beta_0 + i*lbda
        mult = np.exp(-alpha*k[i])/math.pi
        C_T[i] = mult*np.real(epsilon_hat[i]*np.exp(-1j*math.pi*gamma*(i**2)))
    #print(k)
    #print(C_T)
    data = pandas.DataFrame({"Strike": np.exp(k),"Option Premium":C_T})
    print("Option via FFT: For Strike Price %s and the given parameters, we get that the Option Premium is: " %K)
    print(str(round(data.iloc[int(0.5*N)]["Option Premium"],4)))


In [38]:
#Option_Premium_FrFFT(S,r,q,sigma,T,K,n,alpha,eta,lbda)
#For K = 2000
Option_Premium_FrFFT(1900,0.02,0.0187,0.36,0.25,2000,6,0.4,0.25,0.1)
Option_Premium_FrFFT(1900,0.02,0.0187,0.36,0.25,2000,7,1,0.25,0.1)
Option_Premium_FrFFT(1900,0.02,0.0187,0.36,0.25,2000,8,1.4,0.25,0.1)
Option_Premium_FrFFT(1900,0.02,0.0187,0.36,0.25,2000,9,3,0.25,0.1)

Option via FFT: For Strike Price 2000 and the given parameters, we get that the Option Premium is: 
120.773
Option via FFT: For Strike Price 2000 and the given parameters, we get that the Option Premium is: 
47.626
Option via FFT: For Strike Price 2000 and the given parameters, we get that the Option Premium is: 
215.2645
Option via FFT: For Strike Price 2000 and the given parameters, we get that the Option Premium is: 
81.2383


In [39]:
#For K = 2100
Option_Premium_FrFFT(1900,0.02,0.0187,0.36,0.25,2100,6,0.4,0.25,0.1)
Option_Premium_FrFFT(1900,0.02,0.0187,0.36,0.25,2100,7,1,0.25,0.1)
Option_Premium_FrFFT(1900,0.02,0.0187,0.36,0.25,2100,8,1.4,0.25,0.1)
Option_Premium_FrFFT(1900,0.02,0.0187,0.36,0.25,2100,9,3,0.25,0.1)

Option via FFT: For Strike Price 2100 and the given parameters, we get that the Option Premium is: 
105.7646
Option via FFT: For Strike Price 2100 and the given parameters, we get that the Option Premium is: 
32.4225
Option via FFT: For Strike Price 2100 and the given parameters, we get that the Option Premium is: 
184.1533
Option via FFT: For Strike Price 2100 and the given parameters, we get that the Option Premium is: 
64.8482


In [10]:
#For K = 2200
Option_Premium_FrFFT(1900,0.02,0.0187,0.36,0.25,2200,6,0.4,0.25,0.1)
Option_Premium_FrFFT(1900,0.02,0.0187,0.36,0.25,2200,7,1,0.25,0.1)
Option_Premium_FrFFT(1900,0.02,0.0187,0.36,0.25,2200,8,1.4,0.25,0.1)
Option_Premium_FrFFT(1900,0.02,0.0187,0.36,0.25,2200,9,3,0.25,0.1)

Option via FFT: For Strike Price 2200 and the given parameters, we get that the Option Premium is: 
94.8983
Option via FFT: For Strike Price 2200 and the given parameters, we get that the Option Premium is: 
21.4791
Option via FFT: For Strike Price 2200 and the given parameters, we get that the Option Premium is: 
157.5617
Option via FFT: For Strike Price 2200 and the given parameters, we get that the Option Premium is: 
52.7204


In [11]:
result = integrate.quad(lambda x: x, 0, 2)
result

(2.0, 2.220446049250313e-14)

In [12]:
def Fourier_Cosinee(a,b,S,r,q,sigma,T,K):
    def v(S_T,K):
        return Max(S_T-K,0)
    def exp(S_T,S,r,q,sigma,T):
        return np.exp(-0.5*((np.log(S_T)-np.log(S)-(r-q-0.5*sigma**2)*T)/(sigma*math.sqrt(T)))**2)
    def denominator(S_T,sigma,T):
        return sigma*S_T*math.sqrt(2*T*math.pi)
        
    a = integrate.quad(lambda S_T: v(S_T,K)*exp(S_T,S,r,q,sigma,T)/denominator(S_T,sigma,T),np.exp(a),np.exp(b))[0]
    print(a)

Fourier_Cosinee(-12,12,1900,0.02,0.0187,0.36,0.25,2000)
    

95.72411845675965


In [53]:
def okalm(a,b,S,r,q,sigma,T,K,n):
    N = 2**n
    A = np.zeros(N)
    V = np.zeros(N)
    for i in range(N):
        A[i] = 2*np.real(Characteristic(S/K,r,q,sigma,T,(i*math.pi/(b-a)))*np.exp(-1j*i*a*math.pi/(b-a)))/(b-a)
        V[i] = 2*integrate.quad(lambda y: Max(K*(np.exp(y)-1),0)*np.cos((i*math.pi*(y-a))/(b-a)),a,b)[0]/(b-a)
    V[0] = 0.5*V[0]
    result = 0.5*(b-a)*sum(A*V)
    
    print(result)
okalm(-1,1,1900,0.02,0.0187,0.36,0.25,2000,3)


95.59470691976297


In [14]:
result = integrate.quad(lambda x: Max(x-2000,0),np.exp(-8),np.exp(8))
result[0]/8

60142.41076505027

In [15]:
Characteristic(np.log(2100/2000),0.02,0.0187,0.36,0.26,(1*math.pi/2))

(0.05532521596620651+0.9576847098795718j)